In [1]:
!guardrails hub install hub://brainlogic/high_quality_translation -q

/Users/calebcourier/Projects/guardrails/docs/.venv/lib/python3.12/site-packages/torchmetrics/utilities/imports.py:23: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution
Installing hub://brainlogic/high_quality_translation...
/Users/calebcourier/Projects/guardrails/docs/.venv/lib/python3.12/site-packages/torchmetrics/utilities/imports.py:23: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution
/Users/calebcourier/Projects/guardrails/docs/.venv/lib/python3.12/site-packages/transforme

# Translate text with quality checks

**Note:**
To download this example as a Jupyter notebook, click [here](https://github.com/guardrails-ai/guardrails/blob/main/docs/src/examples/translation_with_quality_check.ipynb).

In this example, we will use Guardrails during the translation of a statement from another language to English. We will check whether the translated statement is likely of high quality.

## Objective

We want to translate a statement from different languages to English and ensure that the translated statement accurately reflects the original content.

### Setup

- Install the `unbabel-comet` from source:
  `pip install git+https://github.com/Unbabel/COMET`
- Please accept the model license from:
  https://huggingface.co/Unbabel/wmt22-cometkiwi-da
- Login into Huggingface Hub using:
  huggingface-cli login --token $HUGGINGFACE_TOKEN


In [3]:
! pip install git+https://github.com/Unbabel/COMET -q

In [4]:
from guardrails import Guard
from rich import print
from guardrails.hub import HighQualityTranslation

## Step 1: Define a Guard that uses the validator

This guard will use the HighQualityTranslation validator to validate some string outputs.


We define the prompt and the guard.

In [5]:
prompt = """
Translate the given statement into English:

${statement_to_be_translated}
"""

guard = Guard().use(HighQualityTranslation(on_fail="fix"))

Loading the model Unbabel/wmt22-cometkiwi-da...


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Lightning automatically upgraded your loaded checkpoint from v1.8.2 to v2.5.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../../../.cache/huggingface/hub/models--Unbabel--wmt22-cometkiwi-da/snapshots/1ad785194e391eebc6c53e2d0776cada8f83179a/checkpoints/model.ckpt`
/Users/calebcourier/Projects/guardrails/docs/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Encoder model frozen.
/Users/calebcourier/Projects/guardrails/docs/.venv/lib/python3.12/site-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


## Step 2: Wrap the LLM API call with `Guard`


First, let's try translating a statement that is relatively easy to translate.


In [6]:
# Set your OPENAI_API_KEY as an environment variable
# import os
# os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

statement = "Ich habe keine Ahnung, was ich hier schreiben soll."

res = guard(
    messages=[{"role": "user", "content": prompt}],
    prompt_params={"statement_to_be_translated": statement},
    metadata={"translation_source": statement},
    model="gpt-5-nano",
    max_tokens=1024,
    temperature=1,
)

print(f"Raw LLM Output: {res.raw_llm_output}")
print(f"Validated Output: {res.validated_output}")

15:57:09 - LiteLLM:INFO: utils.py:3389 - 
LiteLLM completion() model= gpt-5-nano; provider = openai

LiteLLM completion() model= gpt-5-nano; provider = openai
15:57:15 - LiteLLM:INFO: utils.py:1282 - Wrapper: Completed Call, calling success_handler
Wrapper: Completed Call, calling success_handler
/Users/calebcourier/Projects/guardrails/docs/.venv/lib/python3.12/site-packages/guardrails/validator_service/__init__.py:84: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/calebcourier/Projects/guardrails/docs/.venv/lib/python3.12/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You ca

Raw LLM Output: I have no idea what I’m supposed to write here.

Validated Output: I have no idea what I’m supposed to write here.

We can look at the logs to see the quality check results:


In [7]:
print(guard.history.last.tree)

Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭─────────────────────────────────────────────── Messages ────────────────────────────────────────────────╮ │
    │ │ ┏━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓                                          │ │
    │ │ ┃ Role ┃ Content                                             ┃                                          │ │
    │ │ ┡━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩                                          │ │
    │ │ │ user │                                                     │                                          │ │
    │ │ │      │ Translate the given statement into English:         │                                          │ │
    │ │ │      │                                                     │                                          │ │
    │ │ │      │ Ich habe keine Ahnung, was ich hier schreiben soll. │                                          │ │
    │ │ │      │                                                     │                                          │ │
    │ │ └──────┴─────────────────────────────────────────────────────┘                                          │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭──────────────────────────────────────────── Raw LLM Output ─────────────────────────────────────────────╮ │
    │ │ I have no idea what I’m supposed to write here.                                                         │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭─────────────────────────────────────────── Validated Output ────────────────────────────────────────────╮ │
    │ │ I have no idea what I’m supposed to write here.                                                         │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

The `guard` wrapper returns the raw LLM response, which is the translated statement and also the validated output. In this case, the translated statement was of a good quality (above the threshold of 0.5), so the validated output is the same as the raw LLM response.

#### Now, let's test with a really low quality translation, and see how Guardrails handles it.


In [8]:
# Parse the code snippet
statement = "अरे भाऊ, आज रात्री जोरदार पार्टी मारूया, जमून टाकूया आणि धमाल करूया!"

## Ideal translation from Marathi -> English:
#  "Hey bro, let's have a great party tonight and have fun!"

output = guard.parse(
    llm_output="It's such a beautiful day, I'm going to the beach.",  ## here, providing a really bad translation
    metadata={"translation_source": statement},
)

# Print the output
print(f"Raw LLM Output: {output.raw_llm_output}")
print(f"Validated Output: {output.validated_output}")

/Users/calebcourier/Projects/guardrails/docs/.venv/lib/python3.12/site-packages/guardrails/validator_service/__init__.py:84: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/calebcourier/Projects/guardrails/docs/.venv/lib/python3.12/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 12.05it/s]


Raw LLM Output: It's such a beautiful day, I'm going to the beach.

Validated Output:

In [9]:
print(guard.history.last.tree)

Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭─────────────────────────────────────────────── Messages ────────────────────────────────────────────────╮ │
    │ │ No messages.                                                                                            │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭──────────────────────────────────────────── Raw LLM Output ─────────────────────────────────────────────╮ │
    │ │ It's such a beautiful day, I'm going to the beach.                                                      │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭─────────────────────────────────────────── Validated Output ────────────────────────────────────────────╮ │
    │ │ ''                                                                                                      │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

As you can see, the translation quality is really bad, and the `HighQualityTranslation` check failed as the translation quality was below the threshold. The validated response is an empty string.

## In this way, you can use Guardrails to ensure that the output of your LLM is of high quality.
